# Data Preprocessing

This notebook is part of the `Fried Chicken Cost Analysis` project and contains the steps taken to clean and transform the webscraped HTML data into structured tabular data. The cleaned data will then be used to build the cost comparison chart.

Goal: Extract ingredients and amounts from web-scrape

# Import Packages and Define Functions

In [18]:
# General data processing
import numpy as np
import pandas as pd

# Packages for pre-processing text
import nltk                       # Natural Language Tool Kit
nltk.download('stopwords')        # For processing stop words (words too common to hold significant meaning)
from nltk.corpus import stopwords # Import above downloaded stopwords
import re                         # Regular Expression
import string                     # For identifying punctuation

# Converting lists and dictionaries stored as strings within DataFrames back to lists and dictionaries
import ast

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Data

In [2]:
# Load the scraped data from allrecipes.com
df = pd.read_csv("../11_raw_data/20231031-2328_scraped_fc_recipes.csv", index_col = 0)

# Examine DataFrame
df.head()

,recipe_url,contents
0,https://www.allrecipes.com/recipe/8805/crispy-...,"{'@context': 'http://schema.org', '@type': ['R..."
1,https://www.allrecipes.com/recipe/8841/oven-fr...,"{'@context': 'http://schema.org', '@type': ['R..."
2,https://www.allrecipes.com/recipe/89268/triple...,"{'@context': 'http://schema.org', '@type': ['R..."
3,https://www.allrecipes.com/recipe/220128/chef-...,"{'@context': 'http://schema.org', '@type': ['R..."
4,https://www.allrecipes.com/recipe/150306/the-b...,"{'@context': 'http://schema.org', '@type': ['R..."


The loaded data consists of 2 columns:
- `recipe_url`: the Uniform Resource Locator(URL) of each recipe.
- `contents`: the data scraped from each recipe's URL stored as a dictionary.

To better understand the scraped data, the keys of the dictionary within `contents` were explicitly examined.

In [8]:
# Examine keys in JSON dictionary containing data within `contents`
sorted_dict_keys = sorted(list(ast.literal_eval(df.loc[0,"contents"]).keys()))

print(f"Dictionary Keys for Scraped Recipe Data:\n")
for index, key in enumerate(sorted_dict_keys):
    print(f"{str(index + 1).rjust(2,'0')}: {key} ")

Dictionary Keys for Scraped Recipe Data:

01: @context 
02: @type 
03: about 
04: aggregateRating 
05: author 
06: cookTime 
07: dateModified 
08: datePublished 
09: description 
10: headline 
11: image 
12: mainEntityOfPage 
13: name 
14: nutrition 
15: prepTime 
16: publisher 
17: recipeCategory 
18: recipeCuisine 
19: recipeIngredient 
20: recipeInstructions 
21: recipeYield 
22: review 
23: totalTime 
24: video 


Specific to material cost analysis, keys `19: recipeIngredient` and `21: recipeYield` are most likely to contain information on the materials used in each recipe and their portioning. This was confirmed below when examining the values associated with each key.

In [135]:
ast.literal_eval(df.loc[3,"contents"])["recipeCuisine"][0]

'Southern'

In [14]:
# Examine recipeIngredient
recipe_name        = ast.literal_eval(df.loc[3,"contents"])["name"]
recipe_portion     = ast.literal_eval(df.loc[3,"contents"])["recipeYield"]
recipe_ingredients = ast.literal_eval(df.loc[3,"contents"])["recipeIngredient"]

print(f"Ingredients for {recipe_name}, yields {recipe_portion} portions.\n")

for index, ing in enumerate(recipe_ingredients):
    print(f"{str(index + 1).rjust(2,'0')}: {ing} ")

Ingredients for Chef John&#39;s Buttermilk Fried Chicken, yields ['4'] portions.

01: 1 (3 1/2) pound chicken, cut into 8 pieces 
02: 1 teaspoon black pepper 
03: 1 teaspoon salt 
04: 1 teaspoon paprika 
05: 0.5 teaspoon white pepper 
06: 0.25 teaspoon dried rosemary 
07: 0.25 teaspoon ground thyme 
08: 0.25 teaspoon dried oregano 
09: 0.25 teaspoon dried sage 
10: 0.25 teaspoon cayenne pepper 
11: 2 cups buttermilk 
12: 2 cups flour 
13: 1 teaspoon salt 
14: 0.5 teaspoon paprika 
15: 0.5 teaspoon cayenne pepper 
16: 0.5 teaspoon garlic powder 
17: 0.5 teaspoon white pepper 
18: 0.5 teaspoon onion powder 
19: 2.5 quarts peanut oil for frying 


Observing Chef John's Buttermilk Fried Chicken recipe's ingredient list:
- ingredient amounts are listed first, followed by the unit of measurement and the ingredient name itself.
- given Allrecipes.com is an American recipe website, the units were assumed to be American imperial units.
- units switch between volume and mass.
- no system to separate ingredients into subprocesses (seasoning the chicken vs. preparing the batter).

Thus, below steps were taken to process the ingredient lists for each recipe:
1) Extract ingredient amounts
2) Extract unit of measurement
3) Extract ingredient name

## Identify only Fried Chicken Recipes

In [119]:
# Keep only recipes, exclude articles, recipe repositories, and others
cond = df["recipe_url"].str.contains("/recipe/")

print(f"Before dropping recipes: {df.shape}")
df = df.loc[cond]
print(f"After dropping recipes: {df.shape}")

Before dropping recipes: (120, 2)
After dropping recipes: (102, 2)


In [148]:
exclude = "|".join(["pan", "bowl", "stir", "rice", "oven", "korea", "japan", "asia", "general", "sandwich", "salad", "ball", "skin", "leg", "chunk", "liver", "drum", "wing", "steak", "breast", "strip", "gizzard", "sauce","loin","thigh","tender","pork","marsala","biryani"])

cond = df["recipe_url"].str.contains(exclude)

for recipe in list(df.loc[~cond,"recipe_url"]):
    print(recipe)

https://www.allrecipes.com/recipe/8805/crispy-fried-chicken/
https://www.allrecipes.com/recipe/89268/triple-dipped-fried-chicken/
https://www.allrecipes.com/recipe/220128/chef-johns-buttermilk-fried-chicken/
https://www.allrecipes.com/recipe/8970/millie-pasquinellis-fried-chicken/
https://www.allrecipes.com/recipe/16573/chicken-fried-chicken/
https://www.allrecipes.com/recipe/8635/southern-fried-chicken/
https://www.allrecipes.com/recipe/24778/better-than-best-fried-chicken/
https://www.allrecipes.com/recipe/55867/baked-bbq-fried-chicken/
https://www.allrecipes.com/recipe/86047/garlic-chicken-fried-chicken/
https://www.allrecipes.com/recipe/15375/fried-chicken-with-creamy-gravy/
https://www.allrecipes.com/recipe/87473/mustard-fried-chicken/
https://www.allrecipes.com/recipe/8802/tanyas-louisiana-southern-fried-chicken/
https://www.allrecipes.com/recipe/8717/deep-south-fried-chicken/
https://www.allrecipes.com/recipe/178809/southern-style-buttermilk-fried-chicken/
https://www.allrecipes

# Extract Ingredients

Each of the above 3 extractions were performed on a single recipe first (Chef John's Buttermilk Fried Chicken) before the same extraction methods were repeated for all recipes. 

## Ingredient Amounts

Ingredient amounts were presented first for each ingredient. Thus, the words within each ingredient were split using whitespace, with the first split being the ingredient amounts.

In [103]:
# Store ingredients in a list
ingredient_list = ast.literal_eval(df.loc[3,"contents"])["recipeIngredient"]

print(f"Ingredient Amounts extracted from Chef John's Buttermilk Fried Chicken\n")

for index, ing in enumerate(ingredient_list):
    print(f"""{str(index + 1).rjust(2,'0')}: {str.split(ing," ")[0].ljust(5," ")} \t {str.split(ing," ")[1:]}""")    

Ingredient Amounts extracted from Chef John's Buttermilk Fried Chicken

01: 1     	 ['(3', '1/2)', 'pound', 'chicken,', 'cut', 'into', '8', 'pieces']
02: 1     	 ['teaspoon', 'black', 'pepper']
03: 1     	 ['teaspoon', 'salt']
04: 1     	 ['teaspoon', 'paprika']
05: 0.5   	 ['teaspoon', 'white', 'pepper']
06: 0.25  	 ['teaspoon', 'dried', 'rosemary']
07: 0.25  	 ['teaspoon', 'ground', 'thyme']
08: 0.25  	 ['teaspoon', 'dried', 'oregano']
09: 0.25  	 ['teaspoon', 'dried', 'sage']
10: 0.25  	 ['teaspoon', 'cayenne', 'pepper']
11: 2     	 ['cups', 'buttermilk']
12: 2     	 ['cups', 'flour']
13: 1     	 ['teaspoon', 'salt']
14: 0.5   	 ['teaspoon', 'paprika']
15: 0.5   	 ['teaspoon', 'cayenne', 'pepper']
16: 0.5   	 ['teaspoon', 'garlic', 'powder']
17: 0.5   	 ['teaspoon', 'white', 'pepper']
18: 0.5   	 ['teaspoon', 'onion', 'powder']
19: 2.5   	 ['quarts', 'peanut', 'oil', 'for', 'frying']


Ingredient amounts were extracted successfully in decimal form.

## Unit of Measurement (UoM)

Next, the units of each ingredient were observed to not necessarily come after ingredient amounts as is the case with ingredient `01`, chicken, in the previous section. Thus, the Natural Language ToolKit(NLTK) was used to standardize the form of each word (singular vs plural) and to exclude stopwords from being picked up. After Porter stemming and stopword removal, each token was checked against a list of common unit of measurements in the American kitchen for extraction.

In [51]:
# Define unit of measurements common to the American home kitchen
measurements = [
    "teaspoon", 
    "tablespoon",
    "cup",
    "quart",
    "pound",
    "ounce"
]

# Define stopwords
eng_stopwords = stopwords.words("english")

# Define a stemmer
stemmer = nltk.stem.PorterStemmer()

print(f"Units of Measurement extracted from Chef John's Buttermilk Fried Chicken\n")

# Iterate through each ingredient
for index, ing in enumerate(ingredient_list):

    # Create variable for printing
    ing_org = ing
    
    # Remove punctuation and take lower case
    for punctuation_mark in string.punctuation:
        ing = ing.replace(punctuation_mark,"").lower()
        
    # Split words into tokens based on whitespace
    tokens = ing.split(" ")

    # Initate blank list to stored stemmed tokens
    stemmed_tokens = []

    # Iterate through all but first token (1st token is ingredient amount)
    for token in tokens[1:]:

        # Exclude stopwords and "", then append stemmed token to blank list
        if (not token in eng_stopwords) and token != "":
            stemmed_tokens.append(stemmer.stem(token))

    # Compare each token to list of common measurements, keeping only those which are units and the first token
    # Each ingredient can only have 1 unit of measurement
    uom = [token for token in stemmed_tokens if token in measurements][0]
    
    print(f"""{str(index + 1).rjust(2,'0')}: {uom} \t was extracted from \t {ing_org}""")   

Units of Measurement extracted from Chef John's Buttermilk Fried Chicken

01: pound 	 was extracted from 	 1 (3 1/2) pound chicken, cut into 8 pieces
02: teaspoon 	 was extracted from 	 1 teaspoon black pepper
03: teaspoon 	 was extracted from 	 1 teaspoon salt
04: teaspoon 	 was extracted from 	 1 teaspoon paprika
05: teaspoon 	 was extracted from 	 0.5 teaspoon white pepper
06: teaspoon 	 was extracted from 	 0.25 teaspoon dried rosemary
07: teaspoon 	 was extracted from 	 0.25 teaspoon ground thyme
08: teaspoon 	 was extracted from 	 0.25 teaspoon dried oregano
09: teaspoon 	 was extracted from 	 0.25 teaspoon dried sage
10: teaspoon 	 was extracted from 	 0.25 teaspoon cayenne pepper
11: cup 	 was extracted from 	 2 cups buttermilk
12: cup 	 was extracted from 	 2 cups flour
13: teaspoon 	 was extracted from 	 1 teaspoon salt
14: teaspoon 	 was extracted from 	 0.5 teaspoon paprika
15: teaspoon 	 was extracted from 	 0.5 teaspoon cayenne pepper
16: teaspoon 	 was extracted from 	 0

The unit of measurement for all 19 ingredients were extracted successfully.

## Ingredient Name

In the third extraction, identifying the core ingredient in a list of words requires a way of assigning importance to each token based on its neighbours. Although this can be achieved using more complex NLP models that take into account word ordering and semantics, a similar method to the previous section was used, whereby each token was compared to a list of common ingredients found in fried chicken recipes.

Furthermore, as ingredients may contain more than 1 word (example: black pepper, white pepper), the strategy used was to first identify if the ingredient contains the common term `pepper`, then to add the matched tokens `white` or `black` to the common term `pepper`, resulting in `black pepper` and `white pepper`. Aside from `pepper`, this strategy was applied to other common terms like `oil` and `powder`.

In [37]:
# Define common ingredients in fried chicken
common_ingredients = [
    "chicken",
    "cayenne",
    "paprika",
    "rosemary",
    "thyme",
    "oregano",
    "sage",
    "buttermilk",
    "salt",
    "flour",
    "onion",
    "garlic",
    "vegetable",
    "peanut",
    "coconut",    
    "white",      # white pepper
    "black"       # black pepper
]

In [88]:
print(f"Ingredients extracted from Chef John's Buttermilk Fried Chicken\n")

# Again iterating through each ingredient 
for index, ing in enumerate(ingredient_list):

    # Create variable for printing
    ing_org = ing
    
    # Remove punctuation and take lower case
    for punctuation_mark in string.punctuation:
        ing = ing.replace(punctuation_mark,"").lower()

    # Split words into tokens based on whitespace
    tokens = ing.split(" ")
    
    # Create blank list to store ingredients
    extracted_ingredients = []

    # Logic for identifying and disambiguation of common ingredients
    # The code here is left explicit for easier reading of logic, a condensed version is used when combining all three extractions
    if "pepper" in tokens:        
        for token in tokens:
            if token in common_ingredients:
                extracted_ingredients.append(token + " pepper")
    elif "powder" in tokens:
        for token in tokens:
            if token in common_ingredients:
                extracted_ingredients.append(token + " powder")
    elif "oil" in tokens:
        for token in tokens:
            if token in common_ingredients:
                extracted_ingredients.append(token + " oil")
    else:
        for token in tokens:
            if token in common_ingredients:
                extracted_ingredients.append(token)
    
    print(f"""{str(index + 1).rjust(2,'0')}: {extracted_ingredients[0].ljust(12," ")} \t was extracted from \t {ing_org}""")   

Ingredients extracted from Chef John's Buttermilk Fried Chicken

01: chicken      	 was extracted from 	 1 (3 1/2) pound chicken, cut into 8 pieces
02: black pepper 	 was extracted from 	 1 teaspoon black pepper
03: salt         	 was extracted from 	 1 teaspoon salt
04: paprika      	 was extracted from 	 1 teaspoon paprika
05: white pepper 	 was extracted from 	 0.5 teaspoon white pepper
06: rosemary     	 was extracted from 	 0.25 teaspoon dried rosemary
07: thyme        	 was extracted from 	 0.25 teaspoon ground thyme
08: oregano      	 was extracted from 	 0.25 teaspoon dried oregano
09: sage         	 was extracted from 	 0.25 teaspoon dried sage
10: cayenne pepper 	 was extracted from 	 0.25 teaspoon cayenne pepper
11: buttermilk   	 was extracted from 	 2 cups buttermilk
12: flour        	 was extracted from 	 2 cups flour
13: salt         	 was extracted from 	 1 teaspoon salt
14: paprika      	 was extracted from 	 0.5 teaspoon paprika
15: cayenne pepper 	 was extracted from

The ingredients were extracted. Note that unlike unit of measurement, stemmification and stopword removal were not performed here. This is due to the possibility of stemmification cutting off letters from ingredients that end with vowels. It is arguable if stopword removal is needed since each token is already being compared to lists of common terms for extraction, and no stopwords exist in those lists, meaning stopwords were already removed anyway.

## Combining Ingredient Amounts, UoM and Ingredients

Finally, all three extractions were combined and condensed into one block of code for brevity.

In [95]:
# Initiate blank dictionary to store ingredients
dict = {
    "recipe_name":[],
    "ing_amt":[],
    "ing_uom":[],
    "ing_name":[]
}

In [96]:
for row in df.loc[[3],:].itertuples():

    # Extract ingredients into a list from JSON dictionary
    ing_list = ast.literal_eval(row[2])["recipeIngredient"]
    recipe_name = ast.literal_eval(row[2])["name"]

    # Iterate through each ingredient
    for ing in ing_list:

        # Append recipe name
        dict["recipe_name"].append(recipe_name)
        
        # Extract ingredient amounts
        try:
            dict["ing_amt"].append(float(ing.split(" ")[0]))
        except:
            dict["ing_amt"].append(np.NaN)
        
        # Remove punctuation and take lower case
        # This step has to come after extracting amount else decimal point will be removed
        for punctuation_mark in string.punctuation:
            ing = ing.replace(punctuation_mark,"").lower()

        # Split string into tokens based on whitespace
        tokens = ing.split(" ")

        # Initate blank list to stored stemmed tokens
        stemmed_tokens = []
    
        # Iterate through all but first token (1st token is ingredient amount)
        for token in tokens[1:]:
    
            # Exclude stopwords and "", then append stemmed token to blank list
            if (not token in eng_stopwords) and token != "":
                stemmed_tokens.append(stemmer.stem(token))
                
        # Extract ingredient UoM
        try:
            dict["ing_uom"].append([uom for uom in stemmed_tokens if uom in measurements][0])
        except:
            dict["ing_uom"].append(np.NaN)

        # Extract ingredient name
        try:
            if "pepper" in tokens:
                dict["ing_name"].append([name + " pepper" for name in tokens if name in common_ingredients][0])
            elif "powder" in tokens:
                dict["ing_name"].append([name + " powder" for name in tokens if name in common_ingredients][0])
            elif "oil" in tokens:
                dict["ing_name"].append([name + " oil" for name in tokens if name in common_ingredients][0])
            else:
                dict["ing_name"].append([name for name in tokens if name in common_ingredients][0])
        except:
            dict["ing_name"].append(np.NaN)

# Convert dictionary into DataFrame
ing_df = pd.DataFrame(dict)

# Examine DataFrame
ing_df

,recipe_name,ing_amt,ing_uom,ing_name
0,Chef John&#39;s Buttermilk Fried Chicken,1.00,pound,chicken
1,Chef John&#39;s Buttermilk Fried Chicken,1.00,teaspoon,black pepper
2,Chef John&#39;s Buttermilk Fried Chicken,1.00,teaspoon,salt
3,Chef John&#39;s Buttermilk Fried Chicken,1.00,teaspoon,paprika
4,Chef John&#39;s Buttermilk Fried Chicken,0.50,teaspoon,white pepper
5,Chef John&#39;s Buttermilk Fried Chicken,0.25,teaspoon,rosemary
6,Chef John&#39;s Buttermilk Fried Chicken,0.25,teaspoon,thyme
7,Chef John&#39;s Buttermilk Fried Chicken,0.25,teaspoon,oregano
8,Chef John&#39;s Buttermilk Fried Chicken,0.25,teaspoon,sage
9,Chef John&#39;s Buttermilk Fried Chicken,0.25,teaspoon,cayenne pepper


The resulting DataFrame contains all 19 ingredients present in Chef John's Buttermilk Fried Chicken recipe. Note index 0, chicken, has a slight problem where instead of extracting 1 chicken of 3.5 pounds in weight, only `1` and `pound` were extracted. This can be addressed by the average weight of 1 whole chicken being roughly 4 pounds, and will be addressed later.

Repeating for all recipes:

In [109]:
# Initiate blank dictionary to store ingredients
dict = {
    "recipe_name":[],
    "recipe_yield": [],
    "ing_amt":[],
    "ing_uom":[],
    "ing_name":[]
}

cond = df["recipe_url"].str.contains("/recipe/")

for row in df.loc[cond].itertuples():

    # Extract ingredients into a list from JSON dictionary
    ing_list = ast.literal_eval(row[2])["recipeIngredient"]
    recipe_name = ast.literal_eval(row[2])["name"]
    recipe_yield = ast.literal_eval(row[2])["recipeYield"][0]

    # Iterate through each ingredient
    for ing in ing_list:

        # Append recipe name and yield
        dict["recipe_name"].append(recipe_name)
        dict["recipe_yield"].append(recipe_yield)
        
        # Extract ingredient amounts
        try:
            dict["ing_amt"].append(float(ing.split(" ")[0]))
        except:
            dict["ing_amt"].append(np.NaN)
        
        # Remove punctuation and take lower case
        # This step has to come after extracting amount else decimal point will be removed
        for punctuation_mark in string.punctuation:
            ing = ing.replace(punctuation_mark,"").lower()

        # Split string into tokens based on whitespace
        tokens = ing.split(" ")

        # Initate blank list to stored stemmed tokens
        stemmed_tokens = []
    
        # Iterate through all but first token (1st token is ingredient amount)
        for token in tokens[1:]:
    
            # Exclude stopwords and "", then append stemmed token to blank list
            if (not token in eng_stopwords) and token != "":
                stemmed_tokens.append(stemmer.stem(token))
                
        # Extract ingredient UoM
        try:
            dict["ing_uom"].append([uom for uom in stemmed_tokens if uom in measurements][0])
        except:
            dict["ing_uom"].append(np.NaN)

        # Extract ingredient name
        try:
            if "pepper" in tokens:
                dict["ing_name"].append([name + " pepper" for name in tokens if name in common_ingredients][0])
            elif "powder" in tokens:
                dict["ing_name"].append([name + " powder" for name in tokens if name in common_ingredients][0])
            elif "oil" in tokens:
                dict["ing_name"].append([name + " oil" for name in tokens if name in common_ingredients][0])
            else:
                dict["ing_name"].append([name for name in tokens if name in common_ingredients][0])
        except:
            dict["ing_name"].append(np.NaN)

In [110]:
# Convert dictionary into DataFrame
ing_df = pd.DataFrame(dict)

In [115]:
cond = ing_df.isna().any(axis = 1)
ing_df.loc[cond,"recipe_name"].nunique()

99

In [116]:
ing_df.loc[cond]

,recipe_name,recipe_yield,ing_amt,ing_uom,ing_name
4,Crispy Fried Chicken,8,NaN,NaN,salt pepper
6,Oven Fried Chicken,6,12.00,NaN,chicken
8,Oven Fried Chicken,6,NaN,NaN,salt pepper
9,Oven Fried Chicken,6,1.00,cup,NaN
10,Oven Fried Chicken,6,3.00,NaN,NaN
...,...,...,...,...,...
1070,Mochiko Asian Fried Chicken,10,4.00,NaN,NaN
1071,Mochiko Asian Fried Chicken,10,0.25,cup,NaN
1073,Mochiko Asian Fried Chicken,10,5.00,NaN,garlic
1076,Mochiko Asian Fried Chicken,10,4.00,NaN,NaN


In [118]:
cond = ing_df.isna().any(axis = 1)
cond2 = ing_df["ing_name"] == "salt pepper"

ing_df.loc[cond & cond2]

,recipe_name,recipe_yield,ing_amt,ing_uom,ing_name
4,Crispy Fried Chicken,8,NaN,NaN,salt pepper
8,Oven Fried Chicken,6,NaN,NaN,salt pepper
55,The Best Chicken Fried Steak,4,NaN,NaN,salt pepper
68,Millie Pasquinelli&#39;s Fried Chicken,12,NaN,NaN,salt pepper
109,Better than Best Fried Chicken,4,NaN,NaN,salt pepper
214,Southern Fried Chicken Livers,4,NaN,NaN,salt pepper
317,Fried Chicken,8,NaN,NaN,salt pepper
345,Honey Fried Chicken,4,NaN,NaN,salt pepper
352,Southern Spicy Fried Chicken,10,NaN,NaN,salt pepper
360,Southern Spicy Fried Chicken,10,NaN,NaN,salt pepper


# Data Enrichment

## Unit Conversion

In [ ]:
# Define metric unit conversions
metric_conversion_rate = {
    # units regarding mass (metric unit gram)
    "pound"     : 453.59233, # https://www.metric-conversions.org/weight/pounds-to-grams.htm

    # units regarding volume (metric unit mL or cm3)
    "teaspoon"  : 4.9289215, # https://www.metric-conversions.org/volume/us-teaspoons-to-milliliters.htm#metricConversionTable?val=1
    "tablespoon": 14.786765, # https://www.metric-conversions.org/volume/us-tablespoons-to-milliliters.htm
    "quart"     : 946.35295, # https://www.metric-conversions.org/volume/us-liquid-quarts-to-milliliters.htm
    "cup"       : 236.58824  # https://www.metric-conversions.org/volume/us-cups-to-milliliters.htm 
}

In [ ]:
# Define metric units
metric_uom = {
    # units regarding mass (metric unit gram)
    "pound"     : "g", # https://www.metric-conversions.org/weight/pounds-to-grams.htm

    # units regarding volume (metric unit mL or cm3)
    "teaspoon"  : "mL", # https://www.metric-conversions.org/volume/us-teaspoons-to-milliliters.htm#metricConversionTable?val=1
    "tablespoon": "mL", # https://www.metric-conversions.org/volume/us-tablespoons-to-milliliters.htm
    "quart"     : "mL", # https://www.metric-conversions.org/volume/us-liquid-quarts-to-milliliters.htm
    "cup"       : "mL"  # https://www.metric-conversions.org/volume/us-cups-to-milliliters.htm 
}

In [ ]:
# # Temp fix, convert chicken ing_amt from 1 pound to 4 pound
cond = ing_df["ing_name"] == "chicken"
ing_df.loc[cond,"ing_amt"] = 4
# ing_df.loc[cond,"ing_name"] = "whole chicken"
ing_df.loc[cond]

In [ ]:
# Map (VLOOKUP) the conversion rates
ing_df["ing_amt_metric"] = ing_df["ing_amt"] * ing_df["ing_uom"].map(metric_conversion_rate)
ing_df["ing_uom_metric"] = ing_df["ing_uom"].map(metric_uom)

In [ ]:
# Examine the conversion result
ing_df

# Load Costs

In [ ]:
cost_df = pd.read_csv("../11_raw_data/20231103-1016_ingredient_cost.csv")

In [ ]:
cost_df.head()

In [ ]:
final_df = ing_df.merge(
    cost_df.loc[:,["Material", "Price\n(CAD)", "Unit", "Density\nMeasurement", "Density\nUnit"]],
    left_on = "ing_name",
    right_on = "Material"
)

In [ ]:
final_df.head()

In [ ]:
final_df["Density\nMeasurement"] = final_df["Density\nMeasurement"].str.replace("-","0").str.replace("","0")
final_df["Density\nMeasurement"] = final_df["Density\nMeasurement"].astype("float")

In [25]:
cost_list = []

for index, row in final_df.iterrows():
    if row["ing_uom_metric"] == row["Unit"]:
        cost_list.append(row["ing_amt_metric"] * row["Price\n(CAD)"])
    else:
        cost_list.append(row["ing_amt_metric"] * row["Price\n(CAD)"] * row["Density\nMeasurement"])

In [26]:
final_df.loc[:,"cost"] = cost_list
final_df

,recipe_name,ing_amt,ing_uom,ing_name,ing_amt_metric,ing_uom_metric,Material,Price\n(CAD),Unit,Density\nMeasurement,Density\nUnit,cost
0,Crispy Fried Chicken,4.00,pound,chicken,1814.369320,g,chicken,0.009980,g,0.0000,-,18.107406
1,Chef John&#39;s Buttermilk Fried Chicken,4.00,pound,chicken,1814.369320,g,chicken,0.009980,g,0.0000,-,18.107406
2,Crispy Fried Chicken,1.00,cup,buttermilk,236.588240,mL,buttermilk,0.004000,mL,0.0000,-,0.946353
3,Chef John&#39;s Buttermilk Fried Chicken,2.00,cup,buttermilk,473.176480,mL,buttermilk,0.004000,mL,0.0000,-,1.892706
4,Crispy Fried Chicken,2.00,cup,flour,473.176480,mL,flour,0.001508,g,0.0503,g/mL,0.035892
5,Chef John&#39;s Buttermilk Fried Chicken,2.00,cup,flour,473.176480,mL,flour,0.001508,g,0.0503,g/mL,0.035892
6,Crispy Fried Chicken,1.00,teaspoon,paprika,4.928922,mL,paprika,0.017500,g,0.0406,g/mL,0.003502
7,Chef John&#39;s Buttermilk Fried Chicken,1.00,teaspoon,paprika,4.928922,mL,paprika,0.017500,g,0.0406,g/mL,0.003502
8,Chef John&#39;s Buttermilk Fried Chicken,0.50,teaspoon,paprika,2.464461,mL,paprika,0.017500,g,0.0406,g/mL,0.001751
9,Crispy Fried Chicken,2.00,quart,vegetable oil,1892.705900,mL,vegetable oil,0.003511,mL,0.0000,-,6.645290


In [27]:
final_df = final_df.groupby(
    by = ["recipe_name","ing_name"],
    as_index = False
).agg(
    cost = ("cost","sum")
)

In [29]:
pivot_df = final_df.pivot(
    columns = "recipe_name",
    index   = "ing_name",
    values = "cost"
)

In [30]:
pivot_df

recipe_name,Chef John&#39;s Buttermilk Fried Chicken,Crispy Fried Chicken
ing_name,,
black pepper,0.005369,NaN
buttermilk,1.892706,0.946353
cayenne pepper,0.002054,NaN
chicken,18.107406,18.107406
flour,0.035892,0.035892
garlic powder,0.002260,NaN
onion powder,0.001518,NaN
oregano,0.002684,NaN
paprika,0.005253,0.003502


In [33]:
pivot_df.columns = ["recipe 1","recipe 2"]
pivot_df

,recipe 1,recipe 2
ing_name,,
black pepper,0.005369,NaN
buttermilk,1.892706,0.946353
cayenne pepper,0.002054,NaN
chicken,18.107406,18.107406
flour,0.035892,0.035892
garlic powder,0.002260,NaN
onion powder,0.001518,NaN
oregano,0.002684,NaN
paprika,0.005253,0.003502


In [78]:
final_df.to_csv("../12_processed_data/recipes_pivot.csv")